# Lecture  Experimental Control 2

### This lesson will review some of the fundamental programming methods required to develop a controlled experiment for Psychological/Neuroscience research

### We will use an auditory experiment as an example but the fundamental methods discussed of controlling the experiment and managing the data explained here will apply to any kind of experiment

### Any experiment has two properties. 
* ### There are some experimental **conditions** that have different stimuli (and/or task instructions) presented
* ### There is a **response** obtained from the subject.  

### The main issues discussed here are:
* ### randomization
* ### obtaining responses from a participant
* ### organizing behavioral data using Pandas DataFrames
* ### saving the data to a file 

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import simpleaudio as sa

### For the purpose of experimental control, we need to start the random number generator. 
### the seed of the random number generator is very important here. 
### If we never change the seed we will always run the same experiment on every participant. 
### We should always use a different seed on different participants.  

In [ ]:
from numpy import random 
rng = random.default_rng(seed = 21)

### I dont want to reinvent the wheel.  Why don't I make use of the functions I wrote in the last lesson. 

In [ ]:
def make_tone(f,duration,sr = 44100,ramp = 200):
    time_vec = np.linspace(0, duration, int(duration*sr)) # The number of samples is the length of time X sampling rate.
    tone = np.sin(f * time_vec  * 2 * np.pi)
    risingramp = np.linspace(0,1,ramp)
    descendingramp = np.linspace(1,0,ramp)
    tone[0:ramp] = tone[0:ramp]*risingramp
    tone[-ramp:] = tone[-ramp:]*descendingramp
    return tone

In [ ]:
def play_sound(tone,volume = 0.05,sr = 44100):
    tone  = tone*32768 / np.max(np.abs(tone)) # scale to the range of the sound card.  
    tone = volume*tone # FOR SAFETY.  PLEASE LIMIT THE MAXIMIM VOLUME! 
    tone  = tone.astype(np.int16) # convert to 16 bit integers. 
    play_obj = sa.play_buffer(tone , 1, 2, sr) # i created an object here. 
    play_obj.wait_done() # tells python to wait for the sound to finish before going any further. 

### For simplicity, I am going to fix certain facts about the auditory examples I am going to use in this tutorial.  

In [ ]:
# fixed facts about the sine functions
duration = 0.25 # length of the sound in seconds. 
volume = 0.005   # DO NOT MAKE VOLUME LARGER THAN 0.5 

### Let's consider a task where the subject has to identify a note played as A (440 Hz) or B (494 Hz).  On each trial, we will present one of the notes, and the subject's task is to respond with A or B on the keyboard. 
### I will implement this task in two different ways here. 

### First let's make the two notes 

In [ ]:
fA = 440 
A_note = make_tone(fA,duration)
fB = 494
B_note = make_tone(fB,duration)

## Randomization  

### To make an experiment that is well controlled, psychologists make use of randomization. 
### For example, if we want to carry out an experiment with different types of emotional stimuli (e.g., words with negative or positive affect), we usually want to present an experimental **condition** (negative or positive) chosen at random on each trial of the experiment 
### As discussed below, there are two ways of doing this - 
* ### selecting a condition at random using a random number generator 
* ### selecting a random order of the stimuli  

### Random Sampling

### In this implementation, I am going to make an experiment that has the number of trials set by the variable ntrials.  
### On each trial, I am going to use rng.integers to randomly select whether to play A or B. 
### I am going to assign the integer 1 to A, and the integer 2 to B.  
### First lets consider the logic of randomly selecting A or B 

In [ ]:
trial_type  = rng.integers(1,3) # This will randomly select a 1 or 2 
print(trial_type)
if trial_type == 1:  # if trial_type is 1 we are going to play A 
    play_sound(A_note,volume=volume)
else:
    play_sound(B_note,volume= volume)

### Run the code block above a few times.  We should see that each time we run it, we select a note at random.  

### We could put that block of code in a for loop and repeat for a certain number of *trials*.  I will set the variable **ntrials** to determine the number of trials. 

### I also need to save what i presented on each trial so I know the correct answer. I am making a **condition** to hold this information 

In [ ]:
ntrials = 6
condition = []
for j in range(ntrials):
    trial_type  = rng.integers(1,3) # This will randomly select a 1 or 2 
    if trial_type == 1:  # if trial_type is 1 we are going to play A 
        play_sound(A_note,volume=volume)
        condition.append('A')
    else:
        play_sound(B_note,volume= volume)
        condition.append('B')
print(condition)

### In principle we could make a small task now, where we play a random note and ask the subject to identify it as A or B.  This type of task is known as a **Two-Alternative Forced Choice** task 
### We just have to collect the responses which we discuss in the next section.  

### Random permutation

### One limitation of the randomization approach presented in the previous section is that we dont have control over the number of trials presented in each condition of the experiment.  

### That is, we will have a different number of A and B notes.   

### In many situations, we want to make sure the number of trials per condition is equal.  In that case, we have to use random **permutation** instead of random sampling as our approach.  

### First consider the following code 

In [ ]:
trial_1 = np.ones(3) # an array with 3 ones
trial_2 = 2*np.ones(3) # an array with 3 twos
trial_order = np.concatenate((trial_1,trial_2)) # concatenate the array
for j in range(ntrials):
    if trial_order[j] == 1:  # if trial_type is 1 we are going to play A 
        play_sound(A_note,volume=volume)
    else:
        play_sound(B_note,volume= volume)
print(trial_order)

### That block of code plays the note A three times and then the note B three times.  The order of trials is controlled by the variable trial_order

### In order to **randomize** the order of presentation, I need to shuffle the entriels of trial_order in a random way. 
### The numpy function, `np.random.permutation` can enable us to do this.  

In [ ]:
trial_1 = np.ones(3) # an array with 3 ones
trial_2 = 2*np.ones(3) # an array with 3 twos
trial_order = np.concatenate((trial_1,trial_2)) # concatenate the array
print(trial_order)
ntrials = np.size(trial_order) #get the length of trial_order
trial_shuffle = np.random.permutation(ntrials)
print(trial_shuffle)
trial_order = trial_order[trial_shuffle]
condition = list()
for j in range(ntrials):
    if trial_order[j] == 1:  # if trial_type is 1 we are going to play A 
        play_sound(A_note,volume=volume)
        condition.append('A')
    else:
        play_sound(B_note,volume= volume)
        condition.append('B')
print(trial_order)
print(condition)

### The advantage of this logic is that we can control the number of trials per condition and equate them.  

## Responses from the Keyboard - `input`

### We can obtain responses from the keyboard using the `input` command

### When you run the next cell, be sure to hit any letter, then the enter key 

### The input command will continue to run until the enter key is pressed.  

In [ ]:
a = input('Type a Key:')

### This is less than ideal for a number of reasons.  For one thing, if you type two letters, it will take them both in.  

### Or if you hit enter before a letter, it will proceed to take an empty response. 

### Nevertheless, for many purposes, it works fine to use the keyboard to collect response information.  



### It can sometimes be useful to control the accepted responses.  

### For example, in the simple experiment above, we may only accept a response of a or b. 

### YOur homework this week addresses how to do this. I will show you one approach here. 

### Let's put it together with the A/B note experiment 

In [ ]:
ntrials_per_condition = 2
trial_1 = np.ones(ntrials_per_condition) # an array with 3 ones
trial_2 = 2*np.ones(ntrials_per_condition) # an array with 3 twos
trial_order = np.concatenate((trial_1,trial_2)) # concatenate the array
ntrials = np.size(trial_order) #get the length of trial_order
shuffle = random.permutation(ntrials) # get a random order of trials
trial_order = trial_order[shuffle] # permute trial order 

condition = list()
trialresponse = list()
for j in range(ntrials):
    if trial_order[j] == 1:  # if trial_type is 1 we are going to play A 
        play_sound(A_note,volume=volume)
        condition.append('A')
    else:
        play_sound(B_note,volume= volume)
        condition.append('B')
    response_check = False # I set the response_check to false.  I will only change response_check if i get a valid response 
    while response_check == False: # This while loop runs until I get a valid response 
        response = input() #Get a keyboard input
        if (response =='a') | (response == 'b'): #check if its an a or b 
            response_check = True # if it is update response_check to true  
            trialresponse.append(response)
        else:
            print('Invalid Response Try Again')  # ask the participant to enter a new response 
print('DONE!')

## Documenting and Saving Experimental Data using Pandas Data frames 

### What do we need to save in our hypothetical experiment? 
### On each trial, we need to know the condition (i.e., the note that was played) and the response (i.e., the note that was identified) and the response

### Let's take a look at the results 

In [ ]:
print(condition)
print(trialresponse)

### We can store them in a pandas DataFrame, and write to a csv or xlsx file.  

In [ ]:
data = pd.DataFrame(columns = ['Condition','Response']) #create a data frame
data['Condition'] = condition #save trial order
data['Response'] = trialresponse  #save response 
#Do one of these
#data.to_csv('A_B.csv')
data.to_excel('A_B.xlsx')